In [26]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

In [14]:
df = pd.read_csv("../../data/processed/tq_data_gridded/2024-09-03_gridded.csv")
df.head()

,ticker,sip_timestamp,ask_price,ask_size,bid_price,bid_size,last_trade_price,last_trade_size,mid_price
0,ADSK,2024-09-03 09:31:00,265.9,1.0,250.01,1.0,259.02,10.0,257.955
1,ADSK,2024-09-03 09:31:10,265.9,1.0,250.01,1.0,259.01,4.0,257.955
2,ADSK,2024-09-03 09:31:20,265.9,1.0,250.01,1.0,259.01,4.0,257.955
3,ADSK,2024-09-03 09:31:30,265.9,1.0,250.01,1.0,259.01,4.0,257.955
4,ADSK,2024-09-03 09:31:40,265.9,1.0,250.01,1.0,259.01,4.0,257.955


In [10]:
datapoints_per_ticker = df.groupby(by="ticker")["sip_timestamp"].apply("count").sort_values(ascending=False)
datapoints_per_ticker.head()

ticker
CB     2340
WBA    2340
UNP    2340
MMM    2339
TXN    2339
Name: sip_timestamp, dtype: int64

In [28]:
ticker = datapoints_per_ticker.index[0]


ticker_data = df.loc[df["ticker"] == ticker, :]

def create_sequences(data, seq_length=10):
    """
    Convert DataFrame into sequences of specified length for LSTM input.
    """
    x, y = [], []
    for i in range(len(data) - seq_length):
        # Create sequences of length `seq_length`
        x.append(data[i:i+seq_length, :-1])  # all columns except the last for features
        y.append(data[i+seq_length, -1])     # last column as target
    return np.array(x), np.array(y)

# Assuming `df` is your input DataFrame
# Convert DataFrame to numpy array
data = ticker_data.to_numpy()

# Create sequences
seq_length = 10  # Define your sequence length
X, y = create_sequences(data, seq_length)

# Model definition
model = Sequential([
    LSTM(50, activation='relu', input_shape=(seq_length, 9)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Print model summary
model.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).